# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [97]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.mlab as mlab
import matplotlib.pyplot as pyplot
import math
import statsmodels as sm 
import pylab
np.random.seed(17)

C1 = '#7fc97f'
C2 = '#beaed4'
C3 = '#fdc086'
C4 = '#ffff99'
C5 = '#386cb0'

%matplotlib inline
pd.set_option('display.max_columns', 500)

In [79]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data_array = data.as_matrix(columns=['call'])

In [16]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [17]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

From simply analyzing the number of callbacks for each group, it is apparent that Whites received more callback than Blacks.

In [18]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [19]:
len(data)

4870

In [20]:
data.groupby('race').count()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
race,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2394,2394,2394,2394,2394,2394,2435,2435,2435,2435,836,861,304,329,1551,959,959,954,959,959,959,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435
w,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2390,2390,2390,2390,2390,2390,2435,2435,2435,2435,836,861,304,329,1551,959,959,954,959,959,959,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435


## 1.) What test is appropriate for this problem? Does CLT apply?
- Since the sample size is above 30 it is appropriate to use the  $z$  statistic. There is a total of 4870 samples, 2435 for each group. With that being said, CLT does apply.

## 2.) What are the null and alternate hypotheses?
**Null Hypotheses**
- Race has no significant impact on the rate of callbacks for resumes.

**Alternate Hypotheses**
- Race has a significant impact on the rate of callbacks for resumes.

In [21]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [82]:
# Creates dataframes for each race
b_data = data[data['race'].str.contains('b')]
w_data = data[data['race'].str.contains('w')]

b_calls = b_data.as_matrix(columns=['call'])
w_calls = w_data.as_matrix(columns=['call'])

Before we begin looking at the call back stats, I wanted to explore some other factors that would deter a callback. This will give us better idea of our two groups. We may find that one group is misrepresented.

In [70]:
def namestr(obj):
    x = [name for name in globals() if globals()[name] is obj]
    y = ''.join(x)
    z = y.title()
    return z

def describe_two (data, data2, column):
    x= data[column].describe()
    y= data2[column].describe()
    print (namestr(data))
    print (x)
    print (namestr(data2))
    print (y)

In [71]:
describe_two(b_data,w_data,'ofjobs')

B_Data
count    2435.000000
mean        3.658316
std         1.219150
min         1.000000
25%         3.000000
50%         4.000000
75%         4.000000
max         7.000000
Name: ofjobs, dtype: float64
W_Data
count    2435.000000
mean        3.664476
std         1.219345
min         1.000000
25%         3.000000
50%         4.000000
75%         4.000000
max         7.000000
Name: ofjobs, dtype: float64


In [72]:
describe_two(b_data,w_data,'yearsexp')

B_Data
count    2435.000000
mean        7.829569
std         5.010764
min         1.000000
25%         5.000000
50%         6.000000
75%         9.000000
max        44.000000
Name: yearsexp, dtype: float64
W_Data
count    2435.000000
mean        7.856263
std         5.079228
min         1.000000
25%         5.000000
50%         6.000000
75%         9.000000
max        26.000000
Name: yearsexp, dtype: float64


In [99]:
describe_two(b_data,w_data,'call')

B_Data
count    2435.000000
mean        0.064476
std         0.245649
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64
W_Data
count    2435.000000
mean        0.096509
std         0.295346
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64


In [73]:
describe_two(b_data,w_data,'education')

B_Data
count    2435.000000
mean        3.616016
std         0.733060
min         0.000000
25%         3.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: education, dtype: float64
W_Data
count    2435.000000
mean        3.620945
std         0.696609
min         0.000000
25%         3.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: education, dtype: float64


In [100]:
describe_two(b_data,w_data,'manager')

B_Data
count    2435.000000
mean        0.151951
std         0.359048
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: manager, dtype: float64
W_Data
count    2435.000000
mean        0.152361
std         0.359446
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: manager, dtype: float64


In [101]:
describe_two(b_data,w_data,'supervisor')

B_Data
count    2435.000000
mean        0.077207
std         0.266977
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: supervisor, dtype: float64
W_Data
count    2435.000000
mean        0.077207
std         0.266977
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: supervisor, dtype: float64


Both groups appear to be represented fairly in this sample.

In [80]:
def ci(sample, interval):
    confidence = interval * .01
    n = len(sample)
    mu = np.mean(sample)
    se = stats.sem(sample)
    h = se * stats.t._ppf((1 + confidence) / 2, n - 1)
    print(namestr(sample), 'mu:', mu)
    print("From this sample, we can be confident that {0:.0f}%".format(
        confidence * 100), 'of the values in', namestr(sample), 'are between:')
    print(float(mu - h), 'and', float(mu + h))
    return

In [81]:
ci(data_array, 95)

Data_Array mu: 0.0804928
From this sample, we can be confident that 95% of the values in Data_Array are between:
0.0728493183851242 and 0.08813631534576416


In [83]:
ci(b_calls, 95)

B_Calls mu: 0.0644764
From this sample, we can be confident that 95% of the values in B_Calls are between:
0.05471454933285713 and 0.07423822581768036


In [85]:
ci(w_calls, 95)

W_Calls mu: 0.0965092
From this sample, we can be confident that 95% of the values in W_Calls are between:
0.08477243036031723 and 0.10824605077505112


## 3.) Compute margin of error, confidence interval, and p-value.

In [93]:
z_crit = stats.norm.ppf(q = .975)
print("z-critical value:")             
print(z_crit)                        

calls_stdev = data_array.std() 

margin_of_error = z_crit * (calls_stdev/math.sqrt(len(data_array)))

print("Margin of Error:")
print(margin_of_error)

confidence_interval = (data_array.mean() - margin_of_error,
                       data_array.mean() + margin_of_error)  

print("Confidence interval:")
print(confidence_interval)

z-critical value:
1.95996398454
Margin of Error:
0.00764081617942
Confidence interval:
(0.072852000686020019, 0.088133633044868348)


In [98]:
sm.stats.weightstats.ztest(w_calls, b_calls)

(array([ 4.11470556], dtype=float32), array([  3.87673945e-05]))

## 4.) Write a story describing the statistical significance in the context or the original problem.

With a p-value of 3.87674 we will **reject** the null hypothesis that race has no significant impact on the rate of callbacks for resumes.
- It in fact does play a significant role in determining whether or not an applicant receives a call back from a potential employer.
- Before assuming that race would be the determining factor, I looked at other columns to see if there were any means with relatively high mean differences similar to the ‘call’ column. There were none.


## 5.) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?



In [105]:
def ztest(column): 
    a = b_data.as_matrix(columns=[column])
    b = w_data.as_matrix(columns=[column])
    z = sm.stats.weightstats.ztest(a, b)
    return z

I could ammend my analysis by running ztest on other columns that would also be considered when hiring an individual.

In [106]:
ztest('education')

(array([-0.24047518]), array([ 0.80996191]))

In [107]:
ztest('ofjobs')

(array([-0.17629293]), array([ 0.86006382]))

In [108]:
ztest('yearsexp')

(array([-0.18461971]), array([ 0.85352733]))